In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

not here


### Bistability

In [2]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [3]:
##### LOAD BOUNDARIES
data_file = 'bi.pickle'
with open(data_file,'rb') as f:
    load_array= pickle.load(f)
exc = load_array[0]
inh = load_array[1]
print(len(exc))

147


In [4]:
t0_array = np.arange(380., -1, -1.)
t_pen = t0_array / 400.

bestControl_1 = [None] * len(t_pen)
bestState_1 = [None] * len(t_pen)
cost_1 = [None] * len(t_pen)
runtime_1 = [None] * len(t_pen)
grad_1 = [None] * len(t_pen)
phi_1 = [None] * len(t_pen)
costnode_1 = [None] * len(t_pen)
weights_1 = [None] * len(t_pen)

In [5]:
initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

cgv_list = [None, "HS", "FR", "PR", "CD", "LS", "DY", "WYL", "HZ", None]

In [11]:
dur_pre = 10
dur_post = 10

i = 50

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-32
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

c_var = [ [0], [1], [0,1]]
p_var = [ [0], [0], [0]]

### CURRENTS
cntrl_vars_0 = [0,1]
prec_vars = [0]

max_I = [3., -3.]
factor_wp = 1. / 0.05
factor_ws = 0.
factor_we = 1.
dur = 400
trans_time = 0.95
    
maxC = [5., -5., 0.18, 0.]

n_dur = int(np.around(dur/aln.params.dt + 1.,1))
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [12]:
init_file = 'control_2.pickle'
final_file_1 = 'control_tradeoff_2b.pickle'

In [13]:
if os.path.isfile(init_file) :
    print("init file found")
    
    with open(init_file,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1[0] = load_array[0][i]
    bestState_1[0] = load_array[1][i]
    cost_1 [0]= load_array[2][i]
    runtime_1[0] = load_array[3][i]
    grad_1 [0]= load_array[4][i]
    phi_1[0] = load_array[5][i]
    costnode_1[0] = load_array[6][i]
    weights_1[0] = load_array[7][i]

init file found
147


In [14]:
if os.path.isfile(final_file_1) :
    print("final file found")
    
    with open(final_file_1,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1 = load_array[0]
    bestState_1 = load_array[1]
    cost_1 = load_array[2]
    runtime_1 = load_array[3]
    grad_1 = load_array[4]
    phi_1 = load_array[5]
    costnode_1 = load_array[6]
    weights_1 = load_array[7]

In [15]:
# get initial parameters and target states

print("------- ", i, exc[i], inh[i])
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.

aln.params.duration = 3000.

control0 = aln.getZeroControl()
control0 = functions.step_control(aln, maxI_ = max_I[0])

aln.run(control=control0)

target_rates = np.zeros((2))
target_rates[0] = aln.rates_exc[0,-1] 
target_rates[1] = aln.rates_inh[0,-1]

control0 = functions.step_control(aln, maxI_ = max_I[1])
aln.run(control=control0)

init_state_vars = np.zeros(( len(state_vars) ))
for j in range(len(state_vars)):
    if aln.state[state_vars[j]].size == 1:
        init_state_vars[j] = aln.state[state_vars[j]][0]
    else:
        init_state_vars[j] = aln.state[state_vars[j]][0,-1]

initVars[i] = init_state_vars

aln.params.duration = dur

target[i] = aln.getZeroTarget()
target[i][:,0,:] = target_rates[0]
target[i][:,1,:] = target_rates[1]

-------  50 0.47500000000000014 0.6000000000000003


In [16]:
# get uncontrolled cost

data.set_parameters(aln)
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.
aln.params.duration = dur

cgv = None

In [19]:
print(np.argmax(np.abs(bestControl_1[0]), axis=2))
print(bestControl_1[0][0,0,1650:1700]/5.)
print(bestControl_1[0][0,0,1674]/5.)

max_percent = (np.argmax(np.abs(bestControl_1[0]), axis=2)[0,0] -100) / 3800
print(max_percent)

[[1598 1546    0    0    0    0]]
[0.15017195 0.14919272 0.14823187 0.14727697 0.14633858 0.14537923
 0.14443629 0.14331204 0.14221236 0.14113332 0.14005669 0.13899877
 0.1379408  0.13688459 0.13584404 0.13482264 0.13382036 0.13279856
 0.1317934  0.13080617 0.12983707 0.12887027 0.1279198  0.12698051
 0.12605777 0.12515148 0.12407316 0.12299684 0.12193738 0.12089338
 0.11983845 0.11879648 0.11776968 0.11675683 0.11574677 0.11475066
 0.11376807 0.11280113 0.1118507  0.11091606 0.10999628 0.10907247
 0.10815928 0.10726081 0.10637676 0.10548422 0.10460328 0.10373605
 0.10288035 0.10202751]
0.12605777100697313
0.39421052631578946


In [20]:
factor_iteration = 1.

##### initial guess
weight_ = 1.
cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

setinit(initVars[i], aln)

weights_1[0] = cost.getParams()

for j in range(1, 381):

    dur = 400. - np.around(j/10., 1)
    T = 4001 - j
    aln.params.duration = dur

    factor_wp = dur / 20.
    cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

    print('-----', j)
    setinit(initVars[i], aln)

    max_it = 3.
    if j > 170:
        max_it = 40.
    elif j > 140:
        max_it = 30.
    elif j > 100.:
        max_it = 20.
    
    if type(bestControl_1[j]) == type(None):
        total_shift = round(j * max_percent * 10)
        prev_shift = round( (j-1) * max_percent * 10)
        shift_ind = total_shift - prev_shift
        #print(total_shift, prev_shift, shift_ind)
        control0 = np.zeros(( bestControl_1[j-1][:,:,100:-101].shape ))
        control0[:,:,:-shift_ind] = 1. * bestControl_1[j-1][:,:,100+shift_ind:-101]
    else:
        if bestControl_1[j].shape[2] == T+200:
            control0 = bestControl_1[j][:,:,100:-100]
        else:
            control0 = bestControl_1[j][:,:,100:-(100+j)]

    bestControl_1[j], bestState_1[j], cost_1[j], runtime_1[j], grad_1[j], phi_1[j], costnode_1[j] = aln.A1(
        control0, target[i], c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = [0,1],
        prec_variables_ = prec_vars, transition_time_ = t_pen[j])

    print(costnode_1[j][0][0][0], costnode_1[j][2][0][1])

    print(costnode_1[0][0][0][0], costnode_1[0][2][0][1])
        
    with open(final_file_1,'wb') as f:
        pickle.dump([bestControl_1, bestState_1, cost_1, runtime_1, grad_1, phi_1,
                costnode_1, weights_1], f)

set cost params:  20.0 1.0 0.0
set cost params:  19.994999999999997 1.0 0.0
----- 1
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  334729.3192492649
Gradient descend method:  None
RUN  1 , total integrated cost =  11.01942994073995
RUN  2 , total integrated cost =  11.019427959902716
RUN  3 , total integrated cost =  11.019427959832004
RUN  3 , total integrated cost =  11.019427959832004
Improved over  3  iterations in  18.279196019000665  seconds by  99.99670795854257  percent.
7.21356118423157e-07 1.9830238921708963
4.058942200395397e-07 1.9830238693113347
set cost params:  19.990000000000002 1.0 0.0
----- 2
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  348987.1014406777
Gradient descend method:  None
RUN  1 , total integrated cost =  11.019439269775763
RUN  2 , total integrated cost =  11.019434890641492
RUN  3 , total integrated cost =  11.019434890478411
RUN  3 , total integrated cost =  11.019434890478411
Improved over  3  iterat